## <span style=color:blue>Illustrations of accessing and using GAEZ data</span>

<span style=color:blue>First some initialization</span>

In [2]:
# These are boiler plate imports that seem useful
# I should probably delete or comment out the ones that aren't used in this script...

import sys
import os
import subprocess
import shutil
# use shutil.rmtree(<path_to_folder>) to do an rm -r on the folder

import json
import csv
import yaml

import urllib

import pandas as pd
import numpy as np

import matplotlib as mpl
from cycler import cycler

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

# sys.path.append('/Users/rick/HACKING/UTILITIES_IN_HACKING/')
# import utils_general


# this one gives a warning - not fixing for now
# from owlready2 import *

# next function is used to generate timestamps that can be used to
# create uniquely named directories and/or files
import datetime
def curr_timestamp():
    current_datetime = datetime.datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    return formatted_datetime


print('Initial imports successful')

Initial imports successful


In [13]:
coordinates = [-122.13333892822266, 39.38333511352539,
-122.13333892822266, 39.29999542236328,
-122.13333892822266, 39.21666717529297,
-122.13333892822266, 39.13333511352539,
-122.13333892822266, 39.04999542236328,
-122.05000305175781, 39.38333511352539,
-122.05000305175781, 39.29999542236328,
-122.05000305175781, 39.21666717529297,
-122.05000305175781, 39.13333511352539,
-122.05000305175781, 39.04999542236328,
-122.05000305175781, 38.96666717529297,
-121.96665954589844, 39.38333511352539,
-121.96665954589844, 39.29999542236328,
-121.96665954589844, 39.21666717529297,
-121.96665954589844, 39.13333511352539,
-121.96665954589844, 39.04999542236328,
-121.96665954589844, 38.96666717529297,
-121.88333892822266, 39.38333511352539,
-121.88333892822266, 39.13333511352539,
-121.88333892822266, 39.04999542236328,
-121.88333892822266, 38.96666717529297,
-121.88333892822266, 38.88333511352539,
-121.80000305175781, 39.38333511352539,
-121.80000305175781, 39.13333511352539,
-121.80000305175781, 39.04999542236328,
-121.80000305175781, 38.96666717529297,
-121.80000305175781, 38.88333511352539,
-121.80000305175781, 38.79999542236328,
-121.71665954589844, 39.38333511352539,
-121.71665954589844, 39.29999542236328,
-121.71665954589844, 39.21666717529297,
-121.71665954589844, 39.13333511352539,
-121.71665954589844, 39.04999542236328,
-121.71665954589844, 38.96666717529297,
-121.71665954589844, 38.88333511352539,
-121.71665954589844, 38.79999542236328,
-121.63333892822266, 39.38333511352539,
-121.63333892822266, 39.29999542236328,
-121.63333892822266, 39.21666717529297,
-121.63333892822266, 39.04999542236328,
-121.63333892822266, 38.96666717529297,
-121.63333892822266, 38.88333511352539,
-121.63333892822266, 38.79999542236328]

### <span style=color:blue>Fetching a .tif file using urllib.</span>

In [3]:
# Using the URL of TIF file Land and Water Resources / Land Cover / Cropland 
url = " https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/res06/I/2010/fdd_2010_har.tif"

# Local directory to save the TIF file in
dir = "/Users/han-yongjin/notebook/ECS289L_FAOSTAT/Project2/"
tif_file_name = "maize_davis.tif"

# Fetch the TIF file using the URL
urllib.request.urlretrieve(url, dir + tif_file_name)

('/Users/han-yongjin/notebook/ECS289L_FAOSTAT/Project2/maize_davis.tif',
 <http.client.HTTPMessage at 0x7f9bb4c8a910>)

### <span style=color:blue>Fetching meta-data about the .tif file using GDAL, in particular the command-line operator gdalinfo.</span>

In [52]:
val = " ".join(["gdalinfo", "-json", dir + tif_file_name])
print(val)
result = subprocess.run([val], shell=True, capture_output=True, text=True)
print(type(result.stdout))
print(result.stdout)
dict = json.loads(result.stdout)
# print(json.dumps(dict, indent=2, sort_keys=True))

gdalinfo -json /Users/han-yongjin/notebook/ECS289L_FAOSTAT/Project2/maize_davis.tif
<class 'str'>
{
  "description":"/Users/han-yongjin/notebook/ECS289L_FAOSTAT/Project2/maize_davis.tif",
  "driverShortName":"GTiff",
  "driverLongName":"GeoTIFF",
  "files":[
    "/Users/han-yongjin/notebook/ECS289L_FAOSTAT/Project2/maize_davis.tif"
  ],
  "size":[
    4320,
    2160
  ],
  "coordinateSystem":{
    "wkt":"GEOGCRS[\"WGS 84\",\n    ENSEMBLE[\"World Geodetic System 1984 ensemble\",\n        MEMBER[\"World Geodetic System 1984 (Transit)\"],\n        MEMBER[\"World Geodetic System 1984 (G730)\"],\n        MEMBER[\"World Geodetic System 1984 (G873)\"],\n        MEMBER[\"World Geodetic System 1984 (G1150)\"],\n        MEMBER[\"World Geodetic System 1984 (G1674)\"],\n        MEMBER[\"World Geodetic System 1984 (G1762)\"],\n        MEMBER[\"World Geodetic System 1984 (G2139)\"],\n        ELLIPSOID[\"WGS 84\",6378137,298.257223563,\n            LENGTHUNIT[\"metre\",1]],\n        ENSEMBLEACCURACY[

In [44]:
def pull_useful(ginfo):  # should give as input the result.stdout from calling gdalinfo -json
    useful = {}
    useful['cornerCoordinates'] = ginfo['cornerCoordinates']
    useful['proj:transform'] = ginfo['stac']['proj:transform']
    useful['size'] = ginfo['size']
    useful['bbox'] = ginfo['stac']['proj:projjson']['bbox']
    useful['espgEncoding'] = ginfo['stac']['proj:epsg']
    return useful


useful = pull_useful(dict)
# print(json.dumps(useful, indent=2, sort_keys=True))

### <span style=color:blue>Fetching value of a pixel using gdallocationinfo.</span>

<span style=color:blue>Note: this .tif has one "Band".  In some casesjl, .tif files have multiple bands, which hold separate parts of the overall data.</span>

In [87]:
# Using the URL of TIF file Land and Water Resources / Land Cover / Cropland 
fodder_crops_url = "https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/res06/T/2010/fdd_2010_har.tif"
sunflower_url = " https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/res06/T/2010/sfl_2010_har.tif"
maize_url = " https://s3.eu-west-1.amazonaws.com/data.gaezdev.aws.fao.org/res06/T/2010/mze_2010_har.tif"

# Local directory to save the TIF file in
dir = "/Users/han-yongjin/notebook/ECS289L_FAOSTAT/Project2/"
tif_file_name = "maize.tif"

# Fetch the TIF file using the URL
urllib.request.urlretrieve(maize_url, dir + tif_file_name)

# converting from lat/long into a pixel location for a global tif with size 4320x2160
def convert_to_pix(lon, lat):
    x = (lon + 180) * 12
    y = (90 - lat) * 12
    return x,y

results = [0 for i in range(len(coordinates)//2)]
for i in range(len(coordinates)//2):
    # x,y = convert_to_pix(-121.8,38.5) # Davis, CA
    x,y = convert_to_pix(coordinates[2*i], coordinates[2*i+1]) # Gemena, D.R. Congo
    # x,y = convert_to_pix(0,0) # in the ocean -- use this to test what the "nodata" value is; turns out it is 0

    val = " ".join(['gdallocationinfo', dir+tif_file_name, str(x), str(y)])

    result = subprocess.run([val], 
                             shell=True, capture_output=True,text=True)

    result_value_idx = result.stdout.find('Value')
    
    results[i] = " ".join([str(coordinates[2*i]), str(coordinates[2*i+1]), result.stdout[result_value_idx + 7:]])
# print(results)
with open('maize_yield_2010.txt', 'w') as file:
    for price in results:
        file.write(price)